<a href="https://colab.research.google.com/github/philsallee/coe_training/blob/master/Classifier_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classifier Comparison

Code Source from Gael Varoquaux and Andreas Muller (from http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)

Modified for Demonstrations by: W. Tod Newman

### Overview
Here we find a comparison of a several classifiers in scikit-learn on synthetic datasets. The point of this example is to illustrate the nature of decision boundaries of different classifiers. This should be taken with a grain of salt, as the intuition conveyed by these examples does not necessarily carry over to real datasets.

Particularly in high-dimensional spaces, data can more easily be separated linearly and the simplicity of classifiers such as naive Bayes and linear SVMs might lead to better generalization than is achieved by other classifiers.

The plots show training points in solid colors and testing points semi-transparent. The lower right shows the classification accuracy on the test set.

### Learning Objectives
*  This example will teach what linear and non-linear classifiers' decision boundaries look like and allow students to evaluate how hyperparameter changes can impact the decision boundaries. 

*  Opportunities to extend this learning include forcing a model to overfit, correcting through simplification of the model or changes to hyperparameters, and also correcting using regularization.


In [ ]:
print(__doc__)


# Code source: Gaël Varoquaux, comment
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause
# Modified for demonstration by Tod Newman

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

Automatically created module for IPython interactive environment


## Model Selection and Evaluation Code

Below we will iterate through a number of different models to determine what the decision boundary between positive and negative examples looks like.  Some of these are very standard models, but all were selected due to their diversity of approach to classification.  

A couple of these models are mature, proven models that are known to generalize well to new data.



In [ ]:
h = .02  # step size in the mesh

# List of Model Names for Display Purposes
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
        "Random Forest", "Tods Random Forest", "Neural Net", "Tods MLP", "AdaBoost",
         "Naive Bayes"]
#####################################################################################################################
#
#  Classifier Definitions Below:
#     THINGS TO TRY:
#     1) Change the value in the KNeighborsClassifier.  See if changing weights to 'distance' makes a difference.
#     2) Change the regularization value (C) of the SVC to see what over- and under-regularization looks like.
#     3) Experiment with hidden layer sizes / number of layers on the MLP Classifier
#     4) Change the optimizer (solver) on the MLPClassifier
#     5) Change the number of estimators on the AdaBoostClassifier.
#     6) See if changing the base_estimator on AdaBoostClassifier helps performance
#     7) Try other things and let us know what you did (put it in your OneNote page)!
######################################################################################################################

#
#  NOTE: If you hover over the classifier name, it usually pops up a window with the definition and
#  parameters of the classifier.
#

######################################################################################################################
#
# Array of Model Definitions Below.  This is where you can make changes.
#
######################################################################################################################
classifiers = [
    KNeighborsClassifier(n_neighbors=5, weights='uniform'), # Tod bumped to 5
    SVC(kernel="linear", C=0.025),
    SVC(kernel='rbf', gamma=6, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    #DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    RandomForestClassifier(n_estimators=211, criterion='gini', bootstrap=False, min_samples_leaf = 4, 
             max_features = 'auto', max_depth = None, warm_start=True), # Tod's General RF Hyperparameters
    MLPClassifier(alpha=1),
    MLPClassifier(solver='adam', alpha=1e-5, shuffle=True, hidden_layer_sizes=(50,30)),
    AdaBoostClassifier(n_estimators=100), # Tod's AdaBoost HP's
    GaussianNB(),
    #QuadraticDiscriminantAnalysis()
    ]

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)

rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

#
# These are scikit-learn datasets that are all fundamentally different.
# This allows us to try our models on different types of data.
#
datasets = [make_moons(noise=0.3, random_state=17),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]

figure = plt.figure(figsize=(27, 9))
i = 1

#
# iterate over datasets, scale the data and split into train/test
#
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)
    
    # Prep mesh grid to allow the plotting of decision boundaries later
    
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Input data")
        
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
               edgecolors='k')
    
    # and testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
               edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].  The desion function 
        # tells us on which side of the hyperplane generated by the classifier 
        # we are (and how far we are away from it). 
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot also the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
                   edgecolors='k')
        # and testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   edgecolors='k', alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

plt.tight_layout()
plt.savefig('classifier_comparison.jpg')
plt.show()